In [ ]:
!pip install transformers torch accelerate bitsandbytes einops gradio typing langchain-community pypdf faiss-cpu sentence-transformers huggingface_hub langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 9.8 MB/s eta 0:00:00
   ━

In [ ]:
from transformers import AutoTokenizer, TextIteratorStreamer, AutoModelForCausalLM
import transformers
import torch
from torch import bfloat16
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from threading import Thread
import asyncio
from huggingface_hub import notebook_login, login
import gradio as gr
from typing import Iterator
import os

In [ ]:
login()  # This will prompt for token when you run it

In [ ]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.chat_template = """{% for message in messages %}
{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}
{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"""

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
MAX_MAX_NEW_TOKENS = 2048
DEFAULT_MAX_NEW_TOKENS = 1024
MAX_INPUT_TOKEN_LENGTH = int(os.getenv("MAX_INPUT_TOKEN_LENGTH", "4096"))

def process_pdfs_sync(files):
    """Process PDF files and create FAISS index"""
    if not files:
        return None

    documents = []
    for file_path in files:
        loader = PyPDFLoader(file_path.name)
        documents.extend(loader.load())

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=128,
        length_function=len,
        is_separator_regex=False,
    )
    chunks = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2",
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
        encode_kwargs={"normalize_embeddings": False}
    )

    return FAISS.from_documents(chunks, embeddings)


In [ ]:
css = """
.spinner {
    display: inline-block;
    width: 50px;
    height: 50px;
    border: 5px solid #f3f3f3;
    border-radius: 50%;
    border-top: 5px solid #3498db;
    animation: spin 1s linear infinite;
}
@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}
"""

In [ ]:
async def process_pdfs(files, status):
    """Process PDFs and reset chat history"""
    # Initial yield with 4 outputs
    yield (
        "<div style='padding: 20px; text-align: center;'><div class='spinner'></div><br>Processing PDFs...</div>",
        gr.update(interactive=False),  # question_input
        gr.update(interactive=False),  # submit_btn
        None  # vector_store (placeholder)
    )

    loop = asyncio.get_event_loop()
    vector_store = await loop.run_in_executor(None, process_pdfs_sync, files)

    # Final yield with 4 outputs
    yield (
        "<div style='color: green; padding: 20px;'>✅ Documents ready</div>",
        gr.update(interactive=True),  # question_input
        gr.update(interactive=True),  # submit_btn
        vector_store  # actual vector_store
    )

In [ ]:
def format_prompt(question: str, context: str, history: list):
    """Create conversation prompt with history"""
    conversation = []
    if context:
        conversation.append({"role": "system", "content": f"You are Medical Ai assistant. Be chearful and hopefull always while responding to the useer. Please don't use any hate speech and any provide response based on the context provided to you. Your response should be short, crisp and concise, so it is easier for the user to understand and implement the solution on their own. Only respond to the queries that are relevant to Medical domain. if user asks anything unrelated to medical domain respond with the message: 'Sorry the question is irrelevant to medical domain, please ask another question'!!!! Use this context: {context}"})

    # Add last 5 exchanges
    for user_msg, bot_resp in history[-5:]:
        conversation.append({"role": "user", "content": user_msg})
        conversation.append({"role": "assistant", "content": bot_resp})

    conversation.append({"role": "user", "content": question})
    return tokenizer.apply_chat_template(conversation, tokenize=False)


In [ ]:
def generate(question: str, vector_store: FAISS, chat_history: list):
    """Generate response with RAG and history"""
    context = ""
    source_info = ""  # To store source metadata

    if vector_store:
        retriever = vector_store.as_retriever(search_kwargs={"k": 3})
        relevant_docs = retriever.invoke(question)
        context = "\n".join([doc.page_content for doc in relevant_docs])

        # Extract source information
        sources = [doc.metadata['source'].split('/')[-1] for doc in relevant_docs]
        pages = [str(doc.metadata.get('page_label', 'N/A')) for doc in relevant_docs]
        source_info = "\n\n📚 Sources:\n" + "\n".join(
            [f"- {source} (Page {page})" for source, page in zip(sources, pages)]
        )

    # Create temporary history copy
    temp_history = chat_history.copy()
    temp_history.append((question, ""))

    # Create prompt with history
    prompt = format_prompt(question, context, chat_history)
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    if input_ids.shape[1] > MAX_INPUT_TOKEN_LENGTH:
        input_ids = input_ids[:, -MAX_INPUT_TOKEN_LENGTH:]
        gr.Warning(f"Trimmed input to {MAX_INPUT_TOKEN_LENGTH} tokens")

    streamer = TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = {
        "input_ids": input_ids,
        "streamer": streamer,
        "max_new_tokens": 1024,
        "do_sample": True,
        "top_p": 0.9,
        "top_k": 50,
        "temperature": 0.6,
        "repetition_penalty": 1.2,
        "eos_token_id": tokenizer.eos_token_id,
    }

    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    output = []
    for token in streamer:
        output.append(token)
        # Update response without sources during streaming
        temp_history[-1] = (question, "".join(output))
        yield temp_history, temp_history

    # After generation completes, add sources if available
    final_response = "".join(output)
    if source_info:
        final_response += source_info

    # Update history with final response
    temp_history[-1] = (question, final_response)
    final_history = temp_history[-5:]

    yield final_history, final_history

In [ ]:
# tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [ ]:
# Add custom CSS for source formatting
css += """
.message-bot {
    white-space: pre-wrap;
    font-family: monospace;
}
.source-info {
    color: #666;
    font-size: 0.9em;
    margin-top: 10px;
    border-left: 3px solid #3498db;
    padding-left: 10px;
}
"""

with gr.Blocks(css=css) as demo:
    vector_store = gr.State()
    chat_history = gr.State([])

    with gr.Row():
        status = gr.HTML("ℹ️ Upload PDFs to begin", elem_classes=["status-text"])

    with gr.Row():
        pdf_upload = gr.UploadButton(
            "📁 Upload PDFs",
            file_types=[".pdf"],
            file_count="multiple",
            variant="primary"
        )

    with gr.Row():
        question_input = gr.Textbox(
            label="Your Medical Question",
            placeholder="Type your health-related question...",
            interactive=False,
            container=False,
        )
        submit_btn = gr.Button("Ask ➤", interactive=False, variant="primary")

    chatbot = gr.Chatbot(
        height=500,
        render_markdown=True,  # Enable markdown rendering
        bubble_full_width=False,
        elem_classes=["message-bot"]
    )

    with gr.Row():
        clear_btn = gr.Button("🧹 Clear Chat History", variant="secondary")
        export_btn = gr.Button("💾 Export Conversation", variant="secondary")

    # PDF processing
    pdf_upload.upload(
        process_pdfs,
        inputs=[pdf_upload, status],
        outputs=[status, question_input, submit_btn, vector_store],
    )

    # Chat handling
    submit_btn.click(
        generate,
        inputs=[question_input, vector_store, chat_history],
        outputs=[chatbot, chat_history],
        api_name="ask_question"
    )

    question_input.submit(
        lambda: gr.update(value=""),
        outputs=[question_input],
        queue=False
    )

    clear_btn.click(
        lambda: ([], None),  # Clear both history and vector store
        outputs=[chat_history, vector_store],
        queue=False
    )

    export_btn.click(
        lambda history: "\n\n".join([f"Q: {q}\nA: {a}" for q, a in history]),
        inputs=[chat_history],
        outputs=[gr.Textbox(label="Exported Conversation", visible=True)],
        queue=False
    )

demo.queue().launch(debug=True)

<ipython-input-14-d2181dea6855>:40: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
<ipython-input-14-d2181dea6855>:40: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8c430c966352c30d4c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-8-489b964da4eb>:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=441bfa505f81c1d0fefd1c803604097c6a9d4b3da42081358bb0f7504d86509e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer
import numpy as np

# Install if needed: !pip install rouge-score

def evaluate_rag(test_cases, vector_store, num_samples=20):
    """
    Evaluate RAG system using ROUGE metrics
    :param test_cases: List of tuples (question, reference_answer)
    :param num_samples: Number of test cases to evaluate (random subset)
    :return: Dictionary with ROUGE scores
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    # Randomly sample test cases
    indices = np.random.choice(len(test_cases), min(num_samples, len(test_cases)))

    for idx in indices:
        question, reference = test_cases[idx]

        # Generate answer using your RAG system
        generated_history = []
        for response, _ in generate(question, vector_store, []):
            pass  # Get final response
        generated_answer = response[-1][1] if response else ""

        # Calculate scores
        score = scorer.score(reference, generated_answer)
        for key in scores:
            scores[key].append(score[key].fmeasure)

    # Return average scores
    return {key: np.mean(values) for key, values in scores.items()}

# Example test cases (replace with your medical QA pairs)
TEST_CASES = [
    ("What are the symptoms of diabetes?",
     "Common symptoms include increased thirst, frequent urination, and blurred vision."),
    ("How is hypertension treated?",
     "Treatment typically involves lifestyle changes and medications like ACE inhibitors.")
]

# Usage (run after processing PDFs):
# vector_store = ...  # Your FAISS index
# scores = evaluate_rag(TEST_CASES, vector_store)
# print(f"ROUGE-1: {scores['rouge1']:.3f}, ROUGE-2: {scores['rouge2']:.3f}, ROUGE-L: {scores['rougeL']:.3f}")

ModuleNotFoundError: No module named 'rouge_score'

In [ ]:
!pip uninstall tensorflow

^C
